# Projet Article 1

In [2]:
import pandas as pd


In [18]:
df_to_moderate=pd.read_csv("get_tweets/selected_tweets.csv")
df_to_moderate.head()

,Id,tweet (without @),source,Harmful,Looked up word
0,1,sale arabe ( arrête avec tes hontes toi ),1.0,1.0,arabe
1,2,y'a une meuf dans classe elle se pleins du rac...,1.0,1.0,arabes
2,3,jparle de qui jveux quand elles se font baiser...,1.0,1.0,baiser
3,4,tellement sexy franchement t'ai baisable ^^,0.0,1.0,baisable
4,5,Je te prend matin midi et soir biatch comme si...,0.0,1.0,biatch


In [19]:
df_to_moderate=df_to_moderate.drop(["Id","source","Looked up word"],axis=1)
df_to_moderate.head()

,tweet (without @),Harmful
0,sale arabe ( arrête avec tes hontes toi ),1.0
1,y'a une meuf dans classe elle se pleins du rac...,1.0
2,jparle de qui jveux quand elles se font baiser...,1.0
3,tellement sexy franchement t'ai baisable ^^,1.0
4,Je te prend matin midi et soir biatch comme si...,1.0


In [12]:
df_article1_messages=pd.read_excel("AmauryModerationAllMessagesInspireFrom3Aout2020.xlsx", sheet_name="ContenuNormal")
df_article1_messages.head()

,_id,content,sender,recipients.0,threadId,timestamp,EchantillonNormal
0,FjuMJ37PjJ7DxqnmT,je te passe mon insta et je te donnerai mon nu...,uoHC9sGdWEkgdazZQ,qHRKFnAqv5ipS9mhR,25woAfTMQDGwuNsoS,1.605613e+12,True
1,Fo7kokj7nDMKD2JWr,pas de soucis ;) je peux te passer mon Whats'a...,uoHC9sGdWEkgdazZQ,qHRKFnAqv5ipS9mhR,25woAfTMQDGwuNsoS,1.605613e+12,True
2,B4nkfsKPjNQSR2AZ8,Ah bah je viens de voir que c’est mal vu de de...,qHRKFnAqv5ipS9mhR,uoHC9sGdWEkgdazZQ,25woAfTMQDGwuNsoS,1.605024e+12,True
3,irioQAHermketHEb5,Salut 👋🏼 ton parcours m’inspire énormément et ...,qHRKFnAqv5ipS9mhR,uoHC9sGdWEkgdazZQ,25woAfTMQDGwuNsoS,1.604848e+12,True
4,ys2Q4j7irtk4S2mYh,Ouii\nEt du coup toi pour la suite tu vas fair...,ku5FNNe97ZiNN3mRv,ptBQ9emHw8he5pMNn,2cDBPgc59itjrnGv4,1.600979e+12,True


In [13]:
df_article1_messages=df_article1_messages.drop(["_id","sender","recipients.0","threadId","timestamp","EchantillonNormal"], axis=1)
df_article1_messages["Harmful"]=0

In [14]:
df_article1_messages.head()

,content,Harmful
0,je te passe mon insta et je te donnerai mon nu...,0
1,pas de soucis ;) je peux te passer mon Whats'a...,0
2,Ah bah je viens de voir que c’est mal vu de de...,0
3,Salut 👋🏼 ton parcours m’inspire énormément et ...,0
4,Ouii\nEt du coup toi pour la suite tu vas fair...,0


In [24]:
print(f"Nombre de tweet à modérer : {df_to_moderate.shape[0]}")

Nombre de tweet à modérer : 220
